In [4]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_groq import ChatGroq

from dotenv import load_dotenv
load_dotenv()

True

In [15]:
tools = [TavilySearchResults(max_results=2)]
model = ChatGroq(model="llama-3.1-70b-versatile").bind_tools(tools=tools)

In [19]:
response = model.invoke([HumanMessage(content="Hello, How are you?")])
print("AI:", response.content)
print("Tool Calls:", response.tool_calls)

AI: I'm just a language model, I don't have feelings or emotions, but I'm functioning properly and ready to assist you. What can I help you with today?
Tool Calls: []


In [20]:
response = model.invoke([HumanMessage(content="What is the weather in hyderabad?")])
print("AI:", response.content)
print("Tool Calls:", response.tool_calls)

AI: 
Tool Calls: [{'name': 'tavily_search_results_json', 'args': {'query': 'Hyderabad weather'}, 'id': 'call_mnw2', 'type': 'tool_call'}]


In [23]:
# building the react agent
from langgraph.prebuilt import create_react_agent

react_agent = create_react_agent(model=model, tools=tools)
react_agent.invoke({"messages": [HumanMessage(content="Hello, How are you?")]})

{'messages': [HumanMessage(content='Hello, How are you?', additional_kwargs={}, response_metadata={}, id='256fc954-54e9-4624-83b4-8251010e8e3f'),
  AIMessage(content="I'm here to help with your questions. What can I assist you with today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 284, 'total_tokens': 302, 'completion_time': 0.072, 'prompt_time': 0.068074413, 'queue_time': 0.005241757, 'total_time': 0.140074413}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b3ae7e594e', 'finish_reason': 'stop', 'logprobs': None}, id='run-033ecefb-a3b0-4795-80f3-b7920bc5fbd7-0', usage_metadata={'input_tokens': 284, 'output_tokens': 18, 'total_tokens': 302})]}

In [24]:
response = react_agent.invoke({
    "messages": [
        HumanMessage(content="What is the weather in hyderabad?"),
    ]
})

response["messages"]    # outputs final response of the agent

[HumanMessage(content='What is the weather in hyderabad?', additional_kwargs={}, response_metadata={}, id='b2589ce1-a2ca-466d-9a5b-e7de73afda1a'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zxt4', 'function': {'arguments': '{"query": "Hyderabad weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 286, 'total_tokens': 306, 'completion_time': 0.08, 'prompt_time': 0.095687015, 'queue_time': 0.005336627999999996, 'total_time': 0.175687015}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b3ae7e594e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-472a0fcb-ca67-4f80-9050-f9cde1c2ca49-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Hyderabad weather'}, 'id': 'call_zxt4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 286, 'output_tokens': 20, 'total_tokens': 306}),
 ToolMessage(content='[{"url": "https://w

In [26]:
# gives a stream of responses with immediate responses
for chunk in react_agent.stream(
    {"messages": [HumanMessage(content="whats the weather in Hyderabad?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_b1qz', 'function': {'arguments': '{"query": "Hyderabad weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 285, 'total_tokens': 305, 'completion_time': 0.080269232, 'prompt_time': 0.073066749, 'queue_time': 0.005620168999999994, 'total_time': 0.153335981}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b3ae7e594e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-077f64d9-4147-444c-9328-3aab7645e918-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Hyderabad weather'}, 'id': 'call_b1qz', 'type': 'tool_call'}], usage_metadata={'input_tokens': 285, 'output_tokens': 20, 'total_tokens': 305})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Hyderabad\', \'region\': \